In [ ]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.optimizers import Adam


In [ ]:
# Load the MovieLens dataset
url = 'https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat'
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
data = pd.read_csv(url, sep='::', header=None, names=columns, engine='python')
movie=["Godfather" , "darknight" ,"LifeofPI" , "3Idoits" , "Slumdog"]
# Map user and movie IDs to contiguous IDs
data['user_id'] = data['user_id'].astype("category").cat.codes.values
data['movie_id'] = data['movie_id'].astype("category").cat.codes.values

# Shuffle the data
data = data.sample(frac=1).reset_index(drop=True)

# Split the data into train and test sets
train_data = data.iloc[:800000]
test_data = data.iloc[800000:]


In [ ]:
class RecommenderAgent:
    def __init__(self, num_users, num_movies, embedding_dim):
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_dim = embedding_dim

        self.user_embedding = Embedding(num_users, embedding_dim, input_length=1)
        self.movie_embedding = Embedding(num_movies, embedding_dim, input_length=1)

        self.flatten = Flatten()
        self.dense = Dense(1)

    def build_model(self):
        user_input = tf.keras.Input(shape=(1,))
        movie_input = tf.keras.Input(shape=(1,))

        user_embedding = self.user_embedding(user_input)
        movie_embedding = self.movie_embedding(movie_input)

        user_embedding = self.flatten(user_embedding)
        movie_embedding = self.flatten(movie_embedding)

        concatenated = tf.keras.layers.Concatenate()([user_embedding, movie_embedding])
        output = self.dense(concatenated)

        model = tf.keras.Model(inputs=[user_input, movie_input], outputs=output)
        model.compile(optimizer=Adam(lr=0.001), loss='mean_squared_error')

        return model

    def train(self, train_data, num_epochs):
        model = self.build_model()
        users = train_data['user_id'].values
        movies = train_data['movie_id'].values
        ratings = train_data['rating'].values

        model.fit([users, movies], ratings, epochs=num_epochs, verbose=1)

        self.model = model

    def recommend(self, user_id, num_recommendations):
        movies = np.arange(self.num_movies)
        user_ids = np.full(self.num_movies, user_id)

        predictions = self.model.predict([user_ids, movies])
        movie_scores = [(movie_id, score) for movie_id, score in enumerate(predictions.flatten())]

        top_movies = sorted(movie_scores, key=lambda x: x[1], reverse=True)[:num_recommendations]
        return top_movies


# Train the agent
num_users = data['user_id'].nunique()
num_movies = data['movie_id'].nunique()
agent = RecommenderAgent(num_users=num_users, num_movies=num_movies, embedding_dim=50)
agent.train(train_data, num_epochs=10)

# Evaluate the agent

print("Mean Squared Error:")


Epoch 1/10
25000/25000 [==============================] - 109s 4ms/step - loss: 4.1254
Epoch 2/10
25000/25000 [==============================] - 99s 4ms/step - loss: 2.1453
Epoch 3/10
25000/25000 [==============================] - 100s 4ms/step - loss: 2.0524
Epoch 4/10
25000/25000 [==============================] - 100s 4ms/step - loss: 2.0028
Epoch 5/10
25000/25000 [==============================] - 99s 4ms/step - loss: 1.9693
Epoch 6/10
25000/25000 [==============================] - 101s 4ms/step - loss: 1.9455
Epoch 7/10
25000/25000 [==============================] - 101s 4ms/step - loss: 1.9264
Epoch 8/10
25000/25000 [==============================] - 101s 4ms/step - loss: 1.9106
Epoch 9/10
25000/25000 [==============================] - 101s 4ms/step - loss: 1.8970
Epoch 10/10
25000/25000 [==============================] - 100s 4ms/step - loss: 1.8865
Mean Squared Error:


In [ ]:
agent = RecommenderAgent(num_users=data['user_id'].nunique(), num_movies=data['movie_id'].nunique(), embedding_dim=50)
agent.train(train_data, num_epochs=10)


Epoch 1/10
25000/25000 [==============================] - 102s 4ms/step - loss: 4.1273
Epoch 2/10
25000/25000 [==============================] - 100s 4ms/step - loss: 2.1465
Epoch 3/10
25000/25000 [==============================] - 98s 4ms/step - loss: 2.0509
Epoch 4/10
25000/25000 [==============================] - 101s 4ms/step - loss: 2.0033
Epoch 5/10
25000/25000 [==============================] - 99s 4ms/step - loss: 1.9707
Epoch 6/10
25000/25000 [==============================] - 99s 4ms/step - loss: 1.9458
Epoch 7/10
25000/25000 [==============================] - 100s 4ms/step - loss: 1.9269
Epoch 8/10
25000/25000 [==============================] - 100s 4ms/step - loss: 1.9113
Epoch 9/10
25000/25000 [==============================] - 100s 4ms/step - loss: 1.8975
Epoch 10/10
25000/25000 [==============================] - 101s 4ms/step - loss: 1.8855


In [ ]:
# Generate a movie recommendation for a specific user
user_id = 0  # User ID for whom we want to recommend a movie
num_recommendations = 5  # Number of movie recommendations to provide

recommendations = agent.recommend(user_id, num_recommendations)
count = 0
# Display the recommended movies

print("Movie Recommendations for User", user_id)
for movie_id, score in recommendations:
    movie_title = data[data['movie_id'] == movie_id].values[0]
    print(movie[count])
    count=count+1
    print("Movie ID:", movie_id, "Title:", movie_title, "Score:", score)


1188/1188 [==============================] - 2s 1ms/step
Movie Recommendations for User 0
Godfather
Movie ID: 32262 Title: [     32083      32262         10 1475603240] Score: 10.8268585
darknight
Movie ID: 31191 Title: [       296      31191         10 1470058087] Score: 10.761883
LifeofPI
Movie ID: 31892 Title: [     39285      31892         10 1452210789] Score: 10.755721
3Idoits
Movie ID: 34104 Title: [     44958      34104         10 1555798656] Score: 10.511042
Slumdog
Movie ID: 29715 Title: [     64345      29715          9 1445556877] Score: 10.081651
